In [ ]:
import dask.dataframe as dd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import r2_score
import dask_ml
import dask
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import QuantileTransformer, PowerTransformer

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*Sending large graph.*")

from dask.distributed import Client, LocalCluster
import dask.multiprocessing

cluster = LocalCluster(processes=True,n_workers=6, threads_per_worker=1)
client = Client(cluster)
import sys
import pickle 

from data_helpers import *
from metrics import *

In [ ]:
cluster = LocalCluster(processes=True,n_workers=6, threads_per_worker=1)
client = Client(cluster)

In [ ]:
folders = [
    'train0_25',
    'train25_50',
    'train50_75',
    'train75_100'
]

# Read Parquet files from each folder into Dask DataFrames
dfs = [dd.read_parquet(folder) for folder in folders]

# Concatenate all DataFrames into a single DataFrame
data = dd.concat(dfs)

In [ ]:
with open('meanDict_allT.pkl', 'rb') as f:
    meanDict = pickle.load(f)

with open('stdDict_allT.pkl', 'rb') as f:
    stdDict = pickle.load(f)

with open('minVal_allT2.pkl', 'rb') as f:
    minDict = pickle.load(f)

with open('zScore_allT2.pkl', 'rb') as f:
    zscoreDict = pickle.load(f)

In [ ]:
f = 'ptend_q0002_26'
nStd = 3
outliers1 = data.loc[((data[f]) > (meanDict[f]+ nStd*stdDict[f])) | ((data[f]) < (meanDict[f]- nStd*stdDict[f]))].compute()

In [ ]:
f = 'ptend_q0002_26'
outliers1['z_score'+'_'+f] = (outliers1[f] - meanDict[f]) / stdDict[f]
outliers1

In [ ]:
f = 'ptend_q0002_55'
nStd = 5
n_outliers5 = data.loc[((data[f]) > (meanDict[f]+ nStd*stdDict[f])) | ((data[f]) < (meanDict[f]- nStd*stdDict[f]))].shape[0].compute()
nStd = 3
n_outliers3 = data.loc[((data[f]) > (meanDict[f]+ nStd*stdDict[f])) | ((data[f]) < (meanDict[f]- nStd*stdDict[f]))].shape[0].compute()
n_data = data.shape[0].compute()

In [ ]:
n_outliers3, n_outliers5, n_data

In [ ]:
f = 'ptend_q0002_55'
nStd = 5
outliers2 = data.loc[((data[f]) > (meanDict[f]+ nStd*stdDict[f])) | ((data[f]) < (meanDict[f]- nStd*stdDict[f]))].compute()


# calc z score

In [ ]:
folders = [
    'train0_25',
    'train25_50',
    'train50_75',
    'train75_100'
]

for folder in folders:
    data = dd.read_parquet(folder)

    for f in ['ptend_q0002_25']:
        data['zscore_'+f] = (data[f] - meanDict[f])/stdDict[f]
    
    data.to_parquet(folder)

In [ ]:
len(allT2)

# check z dict

In [ ]:
data['zscore_ptend_q0002_55'].max().compute(),data['zscore_ptend_q0002_55'].min().compute(),data['zscore_ptend_q0002_26'].max().compute(),data['zscore_ptend_q0002_26'].min().compute()

In [ ]:
data['zscore_ptend_q0002_25'].max().compute(),data['zscore_ptend_q0002_25'].min().compute()

In [ ]:
for f in ['ptend_q0002_24']:
    data['zscore_'+f] = (data[f] - meanDict[f])/stdDict[f]

data['zscore_'+f].min().compute(),data['zscore_'+f].max().compute()

In [ ]:
f = 'ptend_q0002_26'
outliers = data.loc[abs(data['zscore_'+f]) > 5].compute()

# zscore dict

In [ ]:
f = 'ptend_q0002_26'
data.loc[((data['zscore_'+f]) < 5) | ((data['zscore_'+f]) > 5)].shape[0].compute()

In [ ]:
#zscoreDict = dict()
for f in allT2:
    if f not in zscoreDict.keys():
        data['zscore_'+f] = (data[f] - meanDict[f])/stdDict[f]
        zscoreDict[f] = {'min':data['zscore_'+f].min().compute(), 'max':data['zscore_'+f].max().compute()}
        print(f, zscoreDict[f])
    else:
        print('skipped', f)

In [ ]:
zscoreDict

In [ ]:
with open('zScore_allT2.pkl', 'wb') as f:
    pickle.dump(zscoreDict, f)   

In [ ]:
f = 'ptend_q0002_55'
outlier2 = data.loc[abs(data['zscore_'+f]) > 30].compute()

In [ ]:
outlier2